# Epsilon Tutorial
https://symforce.org/tutorials/epsilon_tutorial.html

This notebook describes the epsilon mechanism by which numerical singularities are handled in SymForce. The paper addresses the theory in Section VI, and this tutorial demonstrates the idea through examples.

The basic concept is that it is common to have functions in robotics that are smooth but have singularities at given points. Handwritten functions tend to handle them by adding an if statement at the singularity with some kind of approximation or alternate formulation. This is harder to do with symbolic expressions, and also is not kind to branch prediction. SymForce addresses this with a different method - shifting the input to the function away from the singular point with an infinitesimal variable  (epsilon). This approach is simple and fast for a useful class of removable singularities, with negligible effect to output values for sufficiently small epsilon.

All functions in SymForce that have singularities take epsilon as an argument. In a numerical context, a very small floating point number should be passed in. In the symbolic context, an epsilon symbol should be passed in. Epsilon arguments are currently optional with zero defaults. This is convenient so that playing with expressions in a notebook doesn’t require passing epsilons around. However, this is dangerous and it is extremely important to pass epsilons to get robust behavior or when generating code. Because of this, there are active efforts to make a more intelligent mechanism for the default epsilon to make it less of a footgun to accidentally forget an epsilon and end up with a NaN.

# Goal
We have a function f(x).

In the simplest case, we’re trying to fix a removable singularity at x=0, i.e. f(x).subs(x, 0) == sm.S.NaN

Libraries often do this by checking whether the value of x is close to 0, and using a different method for evaluation there, such as a Taylor expansion. In symbolic expressions, branching like this is messy and expensive.

The idea is to instead make a function f(x, eps) so the value is not NaN, when eps is a small positive number:
f(x, eps).subs(x, 0) != sm.S.NaN
We usually also want that the derivative is not NaN:
f(x, eps).diff(x).subs(x, 0) != NaN
For value continuity we want to match the limit:
f(x, eps).subs(x, 0).limit(eps, 0) == f(x).limit(x, 0)
For derivative continuity we want to match the limit: f(x, eps).diff(x).subs(x, 0).limit(eps, 0) == f(x).diff(x).limit(x, 0)

In [1]:
import numpy as np
import plotly.express as px
import sympy as sm

x = sm.Symbol("x")
eps = sm.Symbol("epsilon")

## An example: sin(x) / x
For the whole section below, let’s pretend x is positive so  is not a fear. We’ll address that later.

In [2]:
# The function `sin(x) / x` looks like this:
def f(x):
    return sm.sin(x) / x


f(x)

sin(x)/x

In [3]:
# And its graph:
x_numerical = np.linspace(-5, 5)
px.line(x=x_numerical, y=np.vectorize(f, otypes=[float])(x_numerical))

In [4]:
# It has a removable singularity at 0, of the form 0/0, which gives NaN:
f(x).subs(x, 0)

nan

In [5]:
# The derivative has the same issue:
f(x).diff(x).subs(x, 0)

nan

One thought to fix this might be to just push the denominator away from 0. This does resolve the NaN, but it produces the wrong value! (Remember, the value at x=0 should be 1)

In [6]:
def f(x, eps):
    return sm.sin(x) / (x + eps)


f(x, eps).subs(x, 0)

0

Similarly, the derivative is wrong, and actually diverges (it should be 0):

In [7]:
f(x, eps).diff(x).subs(x, 0)

1/epsilon

In [8]:
f(x, eps).diff(x).subs(x, 0).limit(eps, 0)

oo

Instead, what we want to do is perturb the input away from the singularity. Effectively, we’re shifting the graph of the function to the left. For removable singularities in well-behaved functions, the error introduced by this is proportional to epsilon. That looks like this:

In [9]:
def f(x, eps):
    x_safe = x + eps
    return sm.sin(x_safe) / x_safe


f(x, eps).subs(x, 0)

sin(epsilon)/epsilon

In [10]:
f(x, eps).subs(x, 0).limit(eps, 0)

1

In [11]:
f(x, eps).diff(x).subs(x, 0)

cos(epsilon)/epsilon - sin(epsilon)/epsilon**2

In [12]:
f(x, eps).diff(x).subs(x, 0).limit(eps, 0)

0

# Automating Verification
We can also write a function that automatically checks that you’ve used epsilon correctly, using SymPy’s ability to take derivatives and limits as we’ve been doing above. That looks something like this - similar functions are available in SymForce in symforce.test_util.epsilon_handling.

In [13]:
def is_epsilon_correct(func, singularity=0, limit_direction="+"):
    """
    Check epsilon handling for a function that accepts a single value and an
    epsilon.

    For epsilon to be handled correctly, the function must:
        1) evaluate to a non-singularity at x=singularity given epsilon
        2) linear approximation of the original must match that taken with
           epsilon then substituted to zero
    """
    # Create symbols
    x = sm.Symbol("x", real=True)
    epsilon = sm.Symbol("epsilon", positive=True)

    is_correct = True

    # Evaluate expression
    expr_eps = func(x, epsilon)
    expr_raw = expr_eps.subs(epsilon, 0)

    display("Expressions (raw / eps):")
    display(expr_raw)
    display(expr_eps)

    # Sub in zero
    expr_eps_at_x_zero = expr_eps.subs(x, singularity)
    if expr_eps_at_x_zero == sm.S.NaN:
        display("[ERROR] Epsilon handling failed, expression at 0 is NaN.")
        is_correct = False

    # Take constant approximation at singularity and check equivalence
    value_x0_raw = sm.simplify(expr_raw.limit(x, singularity, limit_direction))
    value_x0_eps = expr_eps.subs(x, singularity)
    value_x0_eps_sub2 = sm.simplify(value_x0_eps.limit(epsilon, 0))
    if value_x0_eps_sub2 != value_x0_raw:
        display(
            f"[ERROR] Values at x={singularity} do not match (raw / eps / eps.limit):",
        )
        display(value_x0_raw)
        display(value_x0_eps)
        display(value_x0_eps_sub2)
        is_correct = False

    # Take linear approximation at singularity and check equivalence
    derivative_x0_raw = sm.simplify(
        expr_raw.diff(x).limit(x, singularity, limit_direction),
    )
    derivative_x0_eps = expr_eps.diff(x).subs(x, singularity)
    derivative_x0_eps_sub2 = sm.simplify(derivative_x0_eps.limit(epsilon, 0))
    if derivative_x0_eps_sub2 != derivative_x0_raw:
        display(
            f"[ERROR] Derivatives at x={singularity} do not match (raw / eps / eps.limit):",
        )
        display(derivative_x0_raw)
        display(derivative_x0_eps)
        display(derivative_x0_eps_sub2)
        is_correct = False

    return is_correct

## Test sin(x) / x

In [14]:
# Original function fails, singular at x=0
assert is_epsilon_correct(lambda x, eps: sm.sin(x) / x) == False

'Expressions (raw / eps):'

sin(x)/x

sin(x)/x

'[ERROR] Epsilon handling failed, expression at 0 is NaN.'

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

...

nan

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

nan

nan

In [15]:
# Original broken attempt
assert is_epsilon_correct(lambda x, eps: sm.sin(x) / (eps + x)) == False

'Expressions (raw / eps):'

sin(x)/x

sin(x)/(epsilon + x)

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

1

...

0

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

1/epsilon

oo

In [16]:
# Additive on top/bottom works
assert is_epsilon_correct(lambda x, eps: (eps + sm.sin(x)) / (eps + x)) == True

'Expressions (raw / eps):'

sin(x)/x

(epsilon + sin(x))/(epsilon + x)

In [17]:
# Replacing all x with (x + eps) works
assert is_epsilon_correct(lambda x, eps: sm.sin(x + eps) / (x + eps)) == True

'Expressions (raw / eps):'

sin(x)/x

sin(epsilon + x)/(epsilon + x)

## Test (1 - cos(x)) / x

In [18]:
# Original fails, singular at 0
assert is_epsilon_correct(lambda x, eps: (1 - sm.cos(x)) / x) == False

'Expressions (raw / eps):'

(1 - cos(x))/x

(1 - cos(x))/x

'[ERROR] Epsilon handling failed, expression at 0 is NaN.'

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

...

nan

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

1/2

nan

nan

In [19]:
# Value passes if we just replace the denominator, because this one is
# ~ x**2 / x unlike the above which is ~ x / x
assert is_epsilon_correct(lambda x, eps: (1 - sm.cos(x)) / (x + eps)) == False

'Expressions (raw / eps):'

(1 - cos(x))/x

(1 - cos(x))/(epsilon + x)

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

1/2

0

0

In [20]:
# Derivative also passes if we replace both
assert is_epsilon_correct(lambda x, eps: (1 - sm.cos(x + eps)) / (x + eps)) == True

'Expressions (raw / eps):'

(1 - cos(x))/x

(1 - cos(epsilon + x))/(epsilon + x)

## Test x / sqrt(x**2)

In [21]:
# Original fails, singular at 0
assert is_epsilon_correct(lambda x, eps: x / sm.sqrt(x ** 2)) == False

'Expressions (raw / eps):'

x/Abs(x)

x/Abs(x)

'[ERROR] Epsilon handling failed, expression at 0 is NaN.'

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

...

nan

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

nan

nan

In [22]:
# Broken fix #1
assert is_epsilon_correct(lambda x, eps: x / sm.sqrt(x ** 2 + eps ** 2)) == False

'Expressions (raw / eps):'

x/Abs(x)

x/sqrt(epsilon**2 + x**2)

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

1

...

0

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

1/epsilon

oo

In [23]:
# Broken fix #2
assert (
    is_epsilon_correct(
        lambda x, eps: (x + eps) / sm.sqrt(x ** 2 + eps ** 2),
    )
    == False
)

'Expressions (raw / eps):'

x/Abs(x)

(epsilon + x)/sqrt(epsilon**2 + x**2)

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

1/epsilon

oo

In [24]:
# Broken fix #3, ugh
assert (
    is_epsilon_correct(
        lambda x, eps: (x + eps) / (eps + sm.sqrt(x ** 2 + eps ** 2)),
    )
    == False
)

'Expressions (raw / eps):'

x/Abs(x)

(epsilon + x)/(epsilon + sqrt(epsilon**2 + x**2))

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

1

...

1/2

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

1/(2*epsilon)

oo

In [25]:
# Working if you again replace all x with x + eps
assert (
    is_epsilon_correct(
        lambda x, eps: (x + eps) / sm.sqrt((x + eps) ** 2),
    )
    == True
)

'Expressions (raw / eps):'

x/Abs(x)

(epsilon + x)/Abs(epsilon + x)

## Test acos(x) / sqrt(1 - x^2) at 1

In [26]:
# Original fails, singular at 1
assert (
    is_epsilon_correct(
        lambda x, eps: sm.acos(x) / sm.sqrt(1 - x ** 2),
        singularity=1,
    )
    == False
)


'Expressions (raw / eps):'

acos(x)/sqrt(1 - x**2)

acos(x)/sqrt(1 - x**2)

'[ERROR] Epsilon handling failed, expression at 0 is NaN.'

'[ERROR] Values at x=1 do not match (raw / eps / eps.limit):'

...

nan

'[ERROR] Derivatives at x=1 do not match (raw / eps / eps.limit):'

-1/3

nan

nan

In [27]:
# Working if you replace all x with x + eps
assert (
    is_epsilon_correct(
        lambda x, eps: sm.acos(x + eps) / sm.sqrt(1 - (x + eps) ** 2),
        singularity=1,
    )
    == True
)

'Expressions (raw / eps):'

acos(x)/sqrt(1 - x**2)

acos(epsilon + x)/sqrt(1 - (epsilon + x)**2)

## Test atan2(0, x)

In [28]:
# Original is singular
assert is_epsilon_correct(lambda x, eps: sm.atan2(0, x)) == False

'Expressions (raw / eps):'

atan2(0, x)

atan2(0, x)

'[ERROR] Epsilon handling failed, expression at 0 is NaN.'

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

0

nan

nan

In [29]:
# This works
assert is_epsilon_correct(lambda x, eps: sm.atan2(0, x + eps)) == True

'Expressions (raw / eps):'

atan2(0, x)

atan2(0, epsilon + x)

## Handling negative x

In [30]:
def sign_no_zero(x):
    # type: (sf.Scalar) -> sf.Scalar
    """
    Returns -1 if x is negative, 1 if x is positive, and 1 if x is zero (given
    a positive epsilon).
    """
    return 2 * sm.Min(sm.sign(x), 0) + 1

In [31]:
# Test for sin(x) / x
assert (
    is_epsilon_correct(
        lambda x, eps: (sm.sin(x + eps * sign_no_zero(x))) / (x + eps * sign_no_zero(x))
    )
    == True
)

'Expressions (raw / eps):'

sin(x)/x

sin(epsilon*(2*Min(0, sign(x)) + 1) + x)/(epsilon*(2*Min(0, sign(x)) + 1) + x)

In [32]:
# Test for x / sqrt(x**2)
assert (
    is_epsilon_correct(
        lambda x, eps: (x + eps) / sm.sqrt((x + eps) ** 2),
    )
    == True
)

'Expressions (raw / eps):'

x/Abs(x)

(epsilon + x)/Abs(epsilon + x)

In [33]:
# Test for atan2(0, x)
assert (
    is_epsilon_correct(
        lambda x, eps: sm.atan2(0, x + eps * sign_no_zero(x)),
    )
    == True
)


'Expressions (raw / eps):'

atan2(0, x)

atan2(0, epsilon*(2*Min(0, sign(x)) + 1) + x)

## Generalization
So far it seems like for a function  that is singular at , the expression  for a small positive  will be non-singular and retain the same linear approximiation as . So we can easily write a function that does this substitution:

In [34]:
def add_epsilon_sign(expr, var, eps):
    return expr.subs(var, var + eps * sign_no_zero(var))

In [35]:
# Check known example
assert (
    is_epsilon_correct(
        lambda x, eps: add_epsilon_sign(sm.sin(x) / x, x, eps),
    )
    == True
)

'Expressions (raw / eps):'

sin(x)/x

sin(epsilon*(2*Min(0, sign(x)) + 1) + x)/(epsilon*(2*Min(0, sign(x)) + 1) + x)

In [36]:
# Try some more complicated thing nobody wants to epsilon by hand
assert (
    is_epsilon_correct(
        lambda x, eps: add_epsilon_sign(
            (x + sm.sin(x) ** 2) / (x * (1 - 1 / x)),
            x,
            eps,
        )
    )
    == True
)


'Expressions (raw / eps):'

(x + sin(x)**2)/(x*(1 - 1/x))

(epsilon*(2*Min(0, sign(x)) + 1) + x + sin(epsilon*(2*Min(0, sign(x)) + 1) + x)**2)/((1 - 1/(epsilon*(2*Min(0, sign(x)) + 1) + x))*(epsilon*(2*Min(0, sign(x)) + 1) + x))

## Clamping
We could also imagine clamping away from the singularity, instead of shifting with addition. That would look like this:

In [37]:
def add_epsilon_max(expr, var, eps):
    return expr.subs(var, sign_no_zero(var) * sm.Max(eps, sm.Abs(var)))


def add_epsilon_near_1_clamp(expr, var, eps):
    return expr.subs(var, sm.Max(-1 + eps, sm.Min(1 - eps, var)))

In [38]:
assert (
    is_epsilon_correct(
        lambda x, eps: add_epsilon_sign((sm.sin(x) + x ** 2) / x, x, eps),
    )
    == True
)

'Expressions (raw / eps):'

(x**2 + sin(x))/x

((epsilon*(2*Min(0, sign(x)) + 1) + x)**2 + sin(epsilon*(2*Min(0, sign(x)) + 1) + x))/(epsilon*(2*Min(0, sign(x)) + 1) + x)

In [39]:
assert (
    is_epsilon_correct(
        lambda x, eps: add_epsilon_max((sm.sin(x) + x ** 2) / x, x, eps),
    )
    == False
)

'Expressions (raw / eps):'

(x**2*(2*Min(0, sign(x)) + 1)**2 + sin((2*Min(0, sign(x)) + 1)*Abs(x)))/((2*Min(0, sign(x)) + 1)*Abs(x))

((2*Min(0, sign(x)) + 1)**2*Max(epsilon, Abs(x))**2 + sin((2*Min(0, sign(x)) + 1)*Max(epsilon, Abs(x))))/((2*Min(0, sign(x)) + 1)*Max(epsilon, Abs(x)))

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

1

-2*(epsilon**2 + sin(epsilon))*DiracDelta(0)/epsilon + (4*epsilon**2*DiracDelta(0) + 2*epsilon*cos(epsilon)*DiracDelta(0))/epsilon

0

# Case Studies
## Pose2_SE2.from_tangent
Pose2_SE2.from_tangent before epsilon handling looks like:

In [43]:
def pose2_from_tangent(v):
    theta = v[0]
    R = Rot2.from_tangent([theta])

    a = sm.sin(theta) / theta
    b = (1 - sm.cos(theta)) / theta

    t = geo.Vector2(a * v[1] - b * v[2], b * v[1] + a * v[2])
    return geo.Pose2(R, t)

In [44]:
a = sm.sin(theta) / (theta + epsilon)
b = (1 - sm.cos(theta)) / (theta + epsilon)

NameError: name 'theta' is not defined

In [45]:
x, y = sm.symbols("x, y", real=True)

# Looking at a normalization function
epsilon = sm.Symbol("epsilon")
expr = x / sm.sqrt(x ** 2 + y ** 2)
sm.series(expr, x, n=2)
sm.simplify(expr.diff(x).limit(x, 0))


1/Abs(y)

In [46]:
# Simulate a normalization where y = 0
is_epsilon_correct(lambda x, eps: sm.Matrix([x + eps, 0]).normalized()[0])

'Expressions (raw / eps):'

x/Abs(x)

(epsilon + x)/Abs(epsilon + x)

True

In [47]:
is_epsilon_correct(
    lambda x, eps: (x + eps) / sm.sqrt((x + eps) ** 2 + (0 + eps) ** 2),
)

'Expressions (raw / eps):'

x/Abs(x)

(epsilon + x)/sqrt(epsilon**2 + (epsilon + x)**2)

'[ERROR] Values at x=0 do not match (raw / eps / eps.limit):'

1

...

'[ERROR] Derivatives at x=0 do not match (raw / eps / eps.limit):'

0

sqrt(2)/(4*epsilon)

oo

False